In [31]:
from nilmtk import DataSet
import pandas as pd

In [32]:
iawe = DataSet('iawe.h5')

In [33]:
elec = iawe.buildings[1].elec

In [34]:
fridge = elec['fridge']

In [35]:
df = next(fridge.load())

In [36]:
mi = df.columns

In [37]:
new_df = pd.read_csv('ac_seconds4.csv',names =['timestamp','R','A','C','F','V','T'])

In [38]:
import numpy as np
from os.path import join
from nilmtk.datastore import Key
from nilmtk.measurement import LEVEL_NAMES
from nilmtk.utils import check_directory_exists, get_datastore, get_module_directory
from nilm_metadata import convert_yaml_to_hdf5
from copy import deepcopy

In [40]:
# def reindex_fill_na(df, idx):
#     df_copy = deepcopy(df)
#     df_copy = df_copy.reindex(idx)

#     power_columns = [x for x in df.columns if x[0] in ['power']]
#     non_power_columns = [x for x in df.columns if x not in power_columns]

#     for power in power_columns:
#         df_copy[power].fillna(0, inplace=True)
#     for measurement in non_power_columns:
#         df_copy[measurement].fillna(df[measurement].median(), inplace=True)

#     return df_copy
column_mapping = {
    'F': ('frequency', ""),
    'V': ('voltage', ""),
    'T': ('power', 'active'),
    'C': ('current', ''),
    'R': ('power', 'reactive'),
    'A': ('power', 'apparent'),
}

TIMESTAMP_COLUMN_NAME = "timestamp"
TIMEZONE = "Asia/Kolkata"
START_DATETIME, END_DATETIME = '2020-08-12', '2020-08-14'
FREQ = "1T"

output_filename = 'caxe8.h5'

# Open data store
store = get_datastore(output_filename, format='HDF', mode='w')
idx = pd.date_range(start=START_DATETIME, end=END_DATETIME, freq=FREQ)
idx = idx.tz_localize('GMT').tz_convert(TIMEZONE)

key = Key(building=1, meter=1)
print('Loading ', 1)
df =new_df[:]
df.index = pd.to_datetime(df.timestamp.values)
df = df.tz_convert(TIMEZONE)
df = df.drop(TIMESTAMP_COLUMN_NAME, 1)
df.index = pd.to_datetime(df.index.values)
df.columns = pd.MultiIndex.from_tuples(
            [column_mapping[x] for x in df.columns],
            names=LEVEL_NAMES
        )
df = df.apply(pd.to_numeric, errors='ignore')
df = df.dropna()
df = df.astype(np.float32)
df = df.sort_index()
df = df.resample("1T").mean()
df = df.set_index(idx[:115])
# print(df)

assert df.isnull().sum().sum() == 0
# Open data store

store.put(str(key), df)
key = Key(building=1, meter=2)
store.put(str(key), df)
key = Key(building=1, meter=3)
store.put(str(key), df)


################################################################################
store.close()

convert_yaml_to_hdf5('./metadata', output_filename)

print("Done converting iAWE to HDF5!")

Loading  1
Done converting YAML metadata to HDF5!
Done converting iAWE to HDF5!


In [41]:
from combinatorial_optimisation import CO
from fhmm_exact import FHMMExact

In [42]:
experiment1 = {
'power': {'mains': ['apparent','active'],'appliance': ['apparent','active']},
  'sample_rate': 60,
  'appliances': ['air conditioner','fridge'],
  'methods': {"CO":CO({}),"FHMM_EXACT":FHMMExact({'num_of_states':2})},
  'train': {    
    'datasets': {
        'iAWE': {
            'path': './iAWE.h5',
            'buildings': {
                1: {
                    'start_time': '2013-07-13', 
                    'end_time': '2013-08-04'
                    }
                }                
            }
        }
    },
  'test': {
    'datasets': {
        'CAXE': {
            'path': './caxe8.h5',
            'buildings': {
                1: {
                    'start_time': '2020-08-12',
                    'end_time': '2020-08-14'
                    }
                }
            }
        },
        'metrics':['rmse']
    }
}

In [43]:
from api import API

In [44]:
api_results_experiment_1 = API(experiment1)

Joint Testing for all algorithms
Loading data for  CAXE  dataset


C:\Users\Faizan\anaconda3\envs\nilmtk-env\lib\site-packages\nilmtk\metergroup.py:86: RuntimeWarning: Building 1 has an empty 'appliances' list.
  .format(building_id.instance), RuntimeWarning)


KeyError: {'type': 'air conditioner', 'instance': 1}

In [29]:
from nilmtk import DataSet
caxe = DataSet('./caxe7.h5')

In [30]:
caxe.buildings[1].elec

MeterGroup(meters=
  ElecMeter(instance=1, building=1, dataset='CAXE', site_meter, appliances=[])
  MeterGroup(meters=
  )
  MeterGroup(meters=
  )
)